In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

## Screened images

In [3]:
screened_files = list(set(['26_EIPH_566482 L Berliner Blau.svs',
                 '22_EIPH_575216 Berliner Blau.svs',
                 '25_EIPH_568150 Berliner Blau.svs',
                 '20_EIPH_576150 berliner blau.svs',
                 '15_EIPH_568320 berliner blau.svs',
                 '02_EIPH_574162 berliner blau.svs',
                 '18_EIPH_574882 R Berliner Blau.svs',
                 '13_EIPH_570370 Berliner Blau.svs',
                 '16_EIPH_576172 Turnbull blue-002.svs',
                 '19_EIPH_566933 L Berliner Blau.svs',
                 '04_EIPH_567017 Turnbull blue-001.svs',
                 '29_EIPH_566481 L Turnbull blue.svs',
                 '09_EIPH_563417 L berliner blau.svs',
                 '29_EIPH_566481 L Berliner Blau.svs',
                 '20_EIPH_576150 Turnbull blue.svs',
                 '27_EIPH_571557 berliner blau.svs',
                 '11_EIPH_575697 Turnbull blue-002.svs',
                 '31_EIPH_568354 Berliner Blau.svs',
                 '07_EIPH_574882 Berliner Blau-001.svs',

                 '14_EIPH_568381 berliner blau-001.svs',
                 '01_EIPH_563479 Berliner Blau.svs',
                 '08_EIPH_574999 Berliner Blau.svs',
                 '16_EIPH_574882 R Berliner Blau.svs',
                 '17_EIPH_575796 Berliner Blau.svs',
                 '11_EIPH_575697 Berliner Blau.svs',
                 '23_EIPH_563476 Turnbull blue.svs',
                 '01_EIPH_563479 Turnbull blue.svs',
                 '24_EIPH_576255 Turnbull blue.svs',
                 '02_EIPH_574162 Turnbull blue-001.svs',
                 '27_EIPH_571557 Turnbull blue.svs',
                 '10_EIPH_563417 R Berliner Blau.svs',
                 '25_EIPH_568150 Turnbull blue.svs',
                 '21_EIPH_566482 R Turnbull blue.svs',
                 '09_EIPH_563417 L Turnbull blue.svs',
                 '05_EIPH_569923 Berliner Blau.svs',
                 '05_EIPH_569923 Turnbull blue.svs',
                 '04_EIPH_567017 Berliner Blau.svs',

                 '14_EIPH_568381 Turnbull blue.svs',
                 '23_EIPH_563476 Berliner Blau-001.svs',
                 '24_EIPH_576255 Berliner Blau.svs',
                 '30_EIPH_588355 Berliner Blau.svs',
                 '18_EIPH_574882 Turnbull blue-002.svs',
                 '15_EIPH_568320 Turnbull blue.svs',
                 '17_EIPH_575796 Turnbull blue.svs',
                 '30_EIPH_568355 Turnbull blue.svs',
                 '22_EIPH_575216 Turnbull blue.svs',
                 '13_EIPH_570370 Turnbull blue-001.svs',
                 '31_EIPH_568354 Turnbull blue.svs',
                 '03_EIPH_566933 R Berliner Blau neu.svs',
                 '28_EIPH_569948 L berliner blau.svs',
                 '07_EIPH_574882 left lung Turnbull blue.svs',
                 '19_EIPH_566933 L Tunrbull blue.svs',
                 '28_EIPH_569948 L Turnbull blue.svs',
                 '10_EIPH_563417 R Turnbull blue-002.svs',
                 '03_EIPH_566933 R Turnbull blue.svs',
                 
                # Cat
                "BAL_Cat_Turnbull blue_13.svs", 
                "BAL_Cat_Turnbull blue_6.svs", 
                "BAL_Cat_Turnbull blue_1.svs", 
                "BAL_Cat_Turnbull blue_12.svs", 
                "BAL_Cat_Turnbull blue_14.svs", 
                "BAL_Cat_Turnbull blue_10.svs", 
                "BAL_Cat_Turnbull blue_2.svs",


                # Human
                "10835_19 humane BAL Turnbull Blau.svs",
                "10120_19 humane BAL Berliner Blau.svs",
                "10052_19 humane BAL Berliner Blau-001.svs",
                "10080_19 humane BAL Berliner Blau-001.svs",
                "2707_20 TB BAL Human neu.svs",
                "10835_19 humane BAL Berliner Blau.svs",
                "10227_19 humane BAL Berliner Blau.svs",
                "11480_19 humane BAL Berliner Blau.svs",
                "11323_19 humane BAL Berliner Blau.svs",
                "2702_20 BB Human BAL-001.svs",
                "10672_19 humane BAL Berliner Blau.svs",
                "10672_19 humane BAL Turnbull Blau.svs",]))

# Load annotation versions

In [4]:
def set_additional_fields(frame):
    
    frame["width"] = [vector["x2"] - vector["x1"] for vector in frame['vector']]
    frame["height"] = [vector["y2"] - vector["y1"] for vector in frame['vector']]
    frame["scales"] = frame["width"] / frame["height"]
    
    return frame

In [5]:
#species = "Cat", "Equine", "Human"

v0 = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Inference_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Inference_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SDATA_Inference_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SREP_Annotations.pkl")
])
v0 = v0.set_index('unique_identifier')
v0["image_name"] = [name.replace(".tiff", ".svs") for name in v0["image_name"]]
v0 = v0[v0["image_name"].isin(screened_files)]
v0 = set_additional_fields(v0)
print(f"Inference: {len(v0)}")

v_cluster = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Cluster_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Cluster_Annotations.pkl"),
])
v_cluster = v_cluster.set_index('unique_identifier')
v_cluster["image_name"] = [name.replace(".tiff", ".svs") for name in v_cluster["image_name"]]
v_cluster = v_cluster[v_cluster["image_name"].isin(screened_files)]
v_cluster = set_additional_fields(v_cluster)
print(f"Cluster: {len(v_cluster)}")

v_screening = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Screening_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Screening_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SDATA_Screening_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_SREP_Screening_Annotations.pkl"),
])
v_screening = v_screening.set_index('unique_identifier')
v_screening["image_name"] = [name.replace(".tiff", ".svs") for name in v_screening["image_name"]]
v_screening = v_screening[v_screening["image_name"].isin(screened_files)]
v_screening = set_additional_fields(v_screening)
print(f"Screening: {len(v_screening)}")

v_density = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Density_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Density_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SDATA_Density_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_SREP_Density_Annotations.pkl"),
])
v_density = v_density.set_index('unique_identifier')
v_density["image_name"] = [name.replace(".tiff", ".svs") for name in v_density["image_name"]]
v_density = v_density[v_density["image_name"].isin(screened_files)]
print(f"Density: {len(v_density)}")

Inference: 585600
Cluster: 216860
Screening: 303289
Density: 297383


In [7]:
v0.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,user_id,deleted,last_editor,data_set_name,version,width,height,scales
unique_identifier,,,,,,,,,,,,,,,,
875aab61-3f1f-429a-a053-f87aba6c99da,1049539,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 9903.0, 'x2': 10029.0, 'y1': 5508.0, 'y...",1,False,1,SDATA,Inference,126.0,129.0,0.976744
37e8ad72-ba58-4608-aeab-cfdf004d1f00,1049540,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 26435, 'x2': 26544, 'y1': 24003, 'y2': ...",1,True,12,SDATA,Inference,109.0,109.0,1.000000
37f441f4-39ec-478c-9718-c04c4a11c914,1049542,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 18462, 'x2': 18588, 'y1': 10556, 'y2': ...",1,True,12,SDATA,Inference,126.0,126.0,1.000000
74862e27-5881-41d8-90f3-5eb305d35e41,1049545,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 23213.0, 'x2': 23359.0, 'y1': 5401.0, '...",1,True,1,SDATA,Inference,146.0,104.0,1.403846
d718f087-7a46-4e22-8610-ba5f82d04818,1049548,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 12243.0, 'x2': 12375.0, 'y1': 14740.0, ...",1,False,1,SDATA,Inference,132.0,132.0,1.000000


## Global Stats

In [8]:
def get_global_stats(frame, dataset , version, species):
    
    nr_slides = len(frame["image_name"].unique())
    total = frame.shape[0]
    score = int(frame["grade"].mean() * 100)
    _0 = frame[frame["grade"] == 0].shape[0]
    _1 = frame[frame["grade"] == 1].shape[0]
    _2 = frame[frame["grade"] == 2].shape[0]
    _3 = frame[frame["grade"] == 3].shape[0]
    _4 = frame[frame["grade"] == 4].shape[0]
    mean = frame["grade"].mean()
    SD = frame["grade"].std()
    
    frame = [[species, dataset, nr_slides, version, total, score, _0, _1, _2, _3, _4, mean, SD]]
    
    return pd.DataFrame(frame, columns=["species", "dataset" ,"slides", "version", "total", "score", "0", "1", "2", "3", "4", "mean", "SD"])

In [9]:
v0_meta = get_global_stats(v0, "Total", "Inference", "Total")
v_cluster_meta = get_global_stats(v_cluster, "Total", "Cluster", "Total")
v_screening_meta = get_global_stats(v_screening, "Total", "Screening", "Total")
v_density_meta = get_global_stats(v_density, "Total", "Density", "Total")

metaDf = pd.concat([v0_meta, v_cluster_meta, v_screening_meta, v_density_meta])
metaDf.head()

,species,dataset,slides,version,total,score,0,1,2,3,4,mean,SD
0,Total,Total,74,Inference,585600,98,216835,208017,119409,36798,4541,0.982570,0.946059
0,Total,Total,19,Cluster,216860,94,76399,87572,42656,8937,1296,0.944752,0.874400
0,Total,Total,74,Screening,303289,113,92522,109169,74122,24459,3017,1.130466,0.971879
0,Total,Total,74,Density,297383,109,86551,116789,74766,17795,1482,1.094999,0.904234


## Stain 

In [13]:
"BerlinerBlaue: {}".format(len([name for name in v0["image_name"].unique() if "erliner" in name or " BB " in name]))

'BerlinerBlaue: 37'

In [24]:
[name for name in v0["image_name"].unique() if "erliner" in name or " BB " in name]

['2702_20 BB Human BAL-001.svs',
 '10052_19 humane BAL Berliner Blau-001.svs',
 '10080_19 humane BAL Berliner Blau-001.svs',
 '10120_19 humane BAL Berliner Blau.svs',
 '10227_19 humane BAL Berliner Blau.svs',
 '10672_19 humane BAL Berliner Blau.svs',
 '10835_19 humane BAL Berliner Blau.svs',
 '11323_19 humane BAL Berliner Blau.svs',
 '11480_19 humane BAL Berliner Blau.svs',
 '07_EIPH_574882 Berliner Blau-001.svs',
 '08_EIPH_574999 Berliner Blau.svs',
 '09_EIPH_563417 L berliner blau.svs',
 '10_EIPH_563417 R Berliner Blau.svs',
 '13_EIPH_570370 Berliner Blau.svs',
 '14_EIPH_568381 berliner blau-001.svs',
 '15_EIPH_568320 berliner blau.svs',
 '16_EIPH_574882 R Berliner Blau.svs',
 '17_EIPH_575796 Berliner Blau.svs',
 '18_EIPH_574882 R Berliner Blau.svs',
 '19_EIPH_566933 L Berliner Blau.svs',
 '22_EIPH_575216 Berliner Blau.svs',
 '23_EIPH_563476 Berliner Blau-001.svs',
 '24_EIPH_576255 Berliner Blau.svs',
 '25_EIPH_568150 Berliner Blau.svs',
 '28_EIPH_569948 L berliner blau.svs',
 '29_EI

In [18]:
"TurnbullBlaue: {}".format(len([name for name in v0["image_name"].unique() if "erliner" not in name and " BB " not in name]))

'TurnbullBlaue: 37'

In [20]:
[name for name in v0["image_name"].unique() if "erliner" not in name and " BB " not in name]

['BAL_Cat_Turnbull blue_1.svs',
 'BAL_Cat_Turnbull blue_2.svs',
 'BAL_Cat_Turnbull blue_6.svs',
 'BAL_Cat_Turnbull blue_10.svs',
 'BAL_Cat_Turnbull blue_12.svs',
 'BAL_Cat_Turnbull blue_13.svs',
 'BAL_Cat_Turnbull blue_14.svs',
 '2707_20 TB BAL Human neu.svs',
 '10672_19 humane BAL Turnbull Blau.svs',
 '10835_19 humane BAL Turnbull Blau.svs',
 '01_EIPH_563479 Turnbull blue.svs',
 '02_EIPH_574162 Turnbull blue-001.svs',
 '03_EIPH_566933 R Turnbull blue.svs',
 '04_EIPH_567017 Turnbull blue-001.svs',
 '05_EIPH_569923 Turnbull blue.svs',
 '10_EIPH_563417 R Turnbull blue-002.svs',
 '11_EIPH_575697 Turnbull blue-002.svs',
 '13_EIPH_570370 Turnbull blue-001.svs',
 '15_EIPH_568320 Turnbull blue.svs',
 '16_EIPH_576172 Turnbull blue-002.svs',
 '17_EIPH_575796 Turnbull blue.svs',
 '18_EIPH_574882 Turnbull blue-002.svs',
 '19_EIPH_566933 L Tunrbull blue.svs',
 '20_EIPH_576150 Turnbull blue.svs',
 '21_EIPH_566482 R Turnbull blue.svs',
 '23_EIPH_563476 Turnbull blue.svs',
 '24_EIPH_576255 Turnbull b

## Introduced new cells?

In [12]:
188

188

## Images

In [31]:
len(v0["image_name"].unique())

74

# Export MetaData per image and version

In [105]:
def get_image_stats(frame):
    
    #data_set_name	version
    
    images_data = []
    
    for data_set_name in frame["data_set_name"].unique(): 
        
        dfDataSet = frame[frame["data_set_name"]==data_set_name]
        
        for specie in dfDataSet["species"].unique(): 

            specieDf = dfDataSet[dfDataSet["species"]==specie]
            
            if specie == "Cat":
                specie = "feline"

            for file in specieDf["image_name"].unique():

                flieDF = specieDf[specieDf["image_name"] == file]
                
                for version in ['SREP', 'Inference', 'Cluster', 'Screened', 'Density']: 
            
                    versionDf = flieDF[flieDF["version"]==version]
                
                    if len(versionDf) > 0:                       

                        total = versionDf.shape[0]
                        score = int(versionDf["grade"].mean() * 100)
                        _0 = versionDf[versionDf["grade"] == 0].shape[0]
                        _1 = versionDf[versionDf["grade"] == 1].shape[0]
                        _2 = versionDf[versionDf["grade"] == 2].shape[0]
                        _3 = versionDf[versionDf["grade"] == 3].shape[0]
                        _4 = versionDf[versionDf["grade"] == 4].shape[0]
                        mean = versionDf["grade"].mean()
                        SD = versionDf["grade"].std()

                        images_data.append([file, specie, data_set_name, version, total, score, _0, _1, _2, _3, _4, mean, SD])
    
    return pd.DataFrame(images_data, columns=["file", "species", "dataset", "version", "total", "score", "0", "1", "2", "3", "4", "mean", "SD"])

In [106]:
totalDf = pd.concat([v0, v_cluster, v_screening, v_density])

In [107]:
metaImages = get_image_stats(totalDf)

In [108]:
metaImages = metaImages.sort_values(by=["species", "dataset", "file"]) #, "version"
metaImages

,file,species,dataset,version,total,score,0,1,2,3,4,mean,SD
34,10052_19 humane BAL Berliner Blau-001.svs,Human,SDATA,Inference,26787,176,1612,6743,14838,3547,47,1.763841,0.759028
35,10052_19 humane BAL Berliner Blau-001.svs,Human,SDATA,Screened,6351,180,120,1739,3744,739,9,1.807589,0.659142
36,10052_19 humane BAL Berliner Blau-001.svs,Human,SDATA,Density,6271,180,104,1568,4028,569,2,1.808165,0.609628
37,10080_19 humane BAL Berliner Blau-001.svs,Human,SDATA,Inference,20751,133,2419,10142,7094,1064,32,1.332466,0.752932
38,10080_19 humane BAL Berliner Blau-001.svs,Human,SDATA,Screened,6025,146,284,3015,2375,342,9,1.465062,0.682364
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,BAL_Cat_Turnbull blue_2.svs,feline,SDATA,Density,537,93,42,490,5,0,0,0.931099,0.287976
8,BAL_Cat_Turnbull blue_6.svs,feline,SDATA,Inference,12346,9,11132,1213,0,0,1,0.098574,0.299728
9,BAL_Cat_Turnbull blue_6.svs,feline,SDATA,Cluster,11847,6,11084,762,0,0,1,0.064658,0.247982
10,BAL_Cat_Turnbull blue_6.svs,feline,SDATA,Screened,3904,22,3009,895,0,0,0,0.229252,0.420406


In [109]:
metaImages.to_csv("images_meta_data.csv", index=False, sep=";")

## Merge

In [154]:
v_final = v_density.copy()
v_final.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,user_id,deleted,last_editor,data_set_name,version
unique_identifier,,,,,,,,,,,,,
58c6d630-9031-47be-b0f6-a74d8531041d,2643954,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 15005, 'y2': 15055}",1,False,1,SDATA,Density
aadd3497-a7bb-4e91-82b1-b2fbfdc08d6d,2643955,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14945, 'y2': 14995}",1,False,1,SDATA,Density
f535a6f4-9311-43e8-8ff3-674f232dc515,2643956,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14885, 'y2': 14935}",1,False,1,SDATA,Density
c2853048-9036-41cd-8598-dcf2db65f834,2643957,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14825, 'y2': 14875}",1,False,1,SDATA,Density
1b1549b0-4a44-4f12-b167-fdc2f7429843,2643958,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 5, 'x2': 55, 'y1': 14765, 'y2': 14815}",1,False,1,SDATA,Density


In [156]:
vectors = []
for uuid in tqdm(v_final.index.values):
    
    try:
        vectors.append(v_screening.loc[uuid].vector)
    except:
        vectors.append("")

100%|█████████████████████████████████████████████████████████████████████████| 297383/297383 [29:13<00:00, 169.59it/s]


In [157]:
v_final["vector"] = vectors

In [160]:
v_final.to_pickle("../SDATA_Final_Annotations.pkl")

In [163]:
v_final[["species", "image_name", "image_type", "grade", "vector", "data_set_name", "version"]].to_csv("../SDATA_Final_Annotations.csv", sep=";") #index=False, 

## Density changes

In [8]:
screening_changes = {source_uuid: source_grade for source_uuid, source_grade in 
                     zip(v_screening.index, v_screening["grade"])}

density_changes = {density_uuid: density_grade for density_uuid, density_grade in 
                     zip(v_density.index, v_density["grade"])}

In [9]:
total_changes = 0
up_changes = {}
down_changes = {}
grade_one_error = 0

for source_uuid, source_grade in screening_changes.items():
    
    if source_uuid in density_changes:
        
        density_grade = density_changes[source_uuid]
        if source_grade != density_grade:
            name = f"{source_grade} -> {density_grade}"
            
            if source_grade < density_grade:
                if name not in up_changes:
                    up_changes[name] = 0
                up_changes[name] += 1
                total_changes += 1
                
            if source_grade > density_grade:
                if name not in down_changes:
                    down_changes[name] = 0
                down_changes[name] += 1
                total_changes += 1
                
            if abs(density_grade - source_grade) == 1:
                grade_one_error += 1
                
total_changes

38799

In [10]:
grade_one_error

38771

In [11]:
up_changes

{'0 -> 1': 7204,
 '1 -> 2': 5046,
 '2 -> 3': 1179,
 '3 -> 4': 149,
 '0 -> 2': 10,
 '1 -> 3': 2,
 '0 -> 3': 1}

In [12]:
f"Total Up: {sum(up_changes.values())}" 

'Total Up: 13591'

In [13]:
down_changes

{'4 -> 0': 3,
 '1 -> 0': 4309,
 '2 -> 1': 11127,
 '3 -> 2': 8543,
 '4 -> 3': 1214,
 '3 -> 1': 6,
 '4 -> 1': 2,
 '4 -> 2': 2,
 '2 -> 0': 2}

In [14]:
f"Total Down: {sum(down_changes.values())}" 

'Total Down: 25208'